# Projet déforestation 🌴 🌴 🌴

In [1]:
from google.colab import userdata
username = userdata.get('KAGGLE_USERNAME')

## Récupération des clés + téléchargement Kaggle

In [2]:
from google.colab import userdata
import os
# création de variable d'environnement os.environ["KAGGLE_KEY"] et os.environ["KAGGLE_USERNAME"] pour éviter mettre la clé explicitement dans son code.
# Dans le cas, de local, crée un fichier json qui contient les MP et username et accéder au élément correspondant
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [3]:
#Joindre la compétition d'abord dans l'onglet data
!kaggle competitions download -c widsdatathon2019

 99% 458M/460M [00:06<00:00, 63.6MB/s]
100% 460M/460M [00:06<00:00, 77.6MB/s]


In [4]:
!kaggle  --version

Kaggle API 1.5.16


In [5]:
!unzip widsdatathon2019.zip
!unzip -q train_images.zip
!unzip -q leaderboard_test_data.zip
!unzip -q leaderboard_holdout_data.zip

Archive:  widsdatathon2019.zip
  inflating: SampleSubmission.csv    
  inflating: holdout.csv             
  inflating: leaderboard_holdout_data.zip  
  inflating: leaderboard_test_data.zip  
  inflating: solutionFile.csv        
  inflating: testlabels.csv          
  inflating: train_images.zip        
  inflating: traininglabels.csv      


In [6]:
import os
import pathlib

import cv2
import fastai
import pandas as pd

from fastai.vision import *
from fastai.vision.all import *
from fastai.vision.augment import aug_transforms


work_dir = pathlib.Path('./working/')
path = pathlib.Path('.')

train = path / 'train_images'
test = path / 'leaderboard_test_data'
holdout = path / 'leaderboard_holdout_data'
sample_sub = path / 'SampleSubmission.csv'
labels = path / 'traininglabels.csv'

In [7]:
#print(fastai.__version__)

In [8]:
initial_df = pd.read_csv(labels)
initial_df.head()

,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


In [9]:
train_names = [f for f in train.iterdir()]
test_names = [f for f in test.iterdir()]
holdout_names = [f for f in holdout.iterdir()]

In [10]:
to_be_removed = (initial_df
 .assign(new_id=lambda _df: _df['image_id'].str[:-8] + '.jpg')
 .groupby('new_id')
 .agg(palm=('has_oilpalm', 'mean'))
 .query('palm > 0 and palm < 1')
 .index.values)

In [11]:
existing_names = [str(f).split('/')[1] for f in train_names]
len(existing_names), len(set(existing_names) - set(to_be_removed))

(11001, 10414)

In [12]:
df = (initial_df
      .assign(new_id=lambda _df: _df['image_id'].str[:-8] + '.jpg')
      .query('new_id not in @to_be_removed')
      .query('new_id in @existing_names')
      .assign(image_id=lambda _df: _df['new_id'])
      .drop_duplicates(subset=['image_id'])
      .drop(columns='new_id')
      .reset_index(drop=True)
      )
df.shape

(9290, 3)

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

folds = StratifiedKFold(n_splits=11, random_state=2019, shuffle=True)

y = df['has_oilpalm'].values

FOLD = 5 # just one fold for the kaggle kernel

for fold_, (trn_, val_) in enumerate(folds.split(df, y)):
  if fold_==FOLD:
    train_x, trn_y = df.loc[trn_], y[trn_]
    val_x, val_y   = df.loc[val_], y[val_]

    # filename = "resnext101_32x4d_tfms_" + str(fold_)
    filename = "resnet18_tfms_" + str(fold_)
    filename_sub = filename+"_sub.csv"

    print("fold : ", fold_, train_x.shape, val_x.shape, trn_y.mean(), val_y.mean())

    src = ImageDataLoaders.from_df(df.assign(is_valid=lambda _df: np.where(_df.index.isin(val_), True, False)),
                                   str(path), folder=train,
                                   # fn_col=df.columns.get_loc('image_id'),
                                   label_col=df.columns.get_loc('has_oilpalm'),
                                   batch_tfms=aug_transforms(size=256),
                                   valid_col='is_valid')
          # .split_by_idxs(train_x.index, val_x.index)
          # .label_from_df('has_oilpalm')
          # .add_test(test_names + holdout_names))

    # data =  (src.transform(get_transforms(), size=256)
    #        .databunch(bs=16)
    #        .normalize(imagenet_stats))

    # data = (src
    #         .transform(aug_transforms(), size=256)
    #         .databunch(bs=16)
    #         .normalize(imagenet_stats))

    learn = vision_learner(src,
                           models.resnet18,
                           cut=-2,
                           metrics=[accuracy],
                           model_dir=work_dir)
    # learn = create_cnn(data=data,arch= model_f,cut=-2,
    #              metrics=[accuracy],
    #              model_dir='/kaggle/working/models')

    lr = 1e-2
    learn.fit_one_cycle(5, lr)
    lr = 1e-3
    learn.fit_one_cycle(5, lr)

    learn.unfreeze()
    learn.fit_one_cycle(4, slice(1e-5, 1e-4))
    learn.save(filename)
    p, t = learn.tta()

    p = to_np(p); p.shape
    sub = val_x
    # We only recover the probs of having palmoil (column 1)
    sub['preds'] =  p[:, 1]
    sub.to_csv("val_" + filename_sub, index=False)

    # test_dl = learn.dls.test_dl(test_items, with_labels=True)

    # p,t = learn.TTA(ds_type=DatasetType.Test)
    # p,t = learn.TTA(ds_type=DatasetType.Test)
    # p = to_np(p); p.shape
    # ids = np.array([f.name for f in (test_names+holdout_names)]);ids.shape
    # We only recover the probs of having palmoil (column 1)
    # sub = pd.DataFrame(np.stack([ids, p[:,1]], axis=1), columns=df_sample.columns)
    # sub.to_csv(filename_sub, index=False)

    # lets take a second look at the confusion matrix. See if how much we improved.
    interp = ClassificationInterpretation.from_learner(learn)
    interp.plot_confusion_matrix(title='Confusion matrix')

fold :  5 (8445, 3) (845, 3) 0.03457667258732978 0.03550295857988166


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 78.1MB/s]


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [ ]:
from random import randint

def plot_overview(interp:ClassificationInterpretation, classes=['Negative', 'Positive']):
    # top losses will return all validation losses and indexes sorted by the largest first
    tl_val, tl_idx = interp.top_losses()

    #classes = interp.data.classes
    fig, ax = plt.subplots(3, 6, figsize=(24,12))
    fig.suptitle('Predicted / Actual / Loss / Probability',fontsize=20)

    # Random
    for i in range(6):
        random_index = randint(0, len(tl_idx))
        idx = tl_idx[random_index]

        # im, cl = interp.data.dl(DatasetType.Valid).dataset[idx] # fastai V1
        im, cl = interp.dl.dataset[random_index] # fastai V2

        # im = image2np(im.data)

        cl = int(cl)
        ax[0,i].imshow(np.asarray(im))
        ax[0,i].set_xticks([])
        ax[0,i].set_yticks([])
        #ax[0,i].set_title(f' ? / ? / {interp.losses[idx]:.2f} / ?')
        #ax[0,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[0,0].set_ylabel('Random samples', fontsize=16, rotation=0, labelpad=80)

    # Most incorrect or top losses
    for i in range(6):
        idx = int(tl_idx[i])
        # im,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        im, cl = interp.dl.dataset[idx] # fastai V2
        cl = int(cl)
        # im = image2np(im.data)
        ax[1,i].imshow(np.asarray(im))
        ax[1,i].set_xticks([])
        ax[1,i].set_yticks([])
        # ax[1,i].set_title(f' ? / ? / {interp.losses[idx]:.2f} / ?')
        # ax[1,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[1,0].set_ylabel('Most incorrect\nsamples', fontsize=16, rotation=0, labelpad=80)

    # Most correct or least losses
    for i in range(6):
        idx = int(tl_idx[len(tl_idx) - i - 1])
        # im,cl = interp.data.dl(DatasetType.Valid).dataset[idx]
        im, cl = interp.dl.dataset[idx] # fastai V2
        cl = int(cl)
        # im = image2np(im.data)
        ax[2,i].imshow(np.asarray(im))
        ax[2,i].set_xticks([])
        ax[2,i].set_yticks([])
        # ax[2,i].set_title(f' ? / ? / {interp.losses[idx]:.2f} / ?')
        # ax[2,i].set_title(f'{classes[interp.pred_class[idx]]} / {classes[cl]} / {interp.losses[idx]:.2f} / {interp.probs[idx][cl]:.2f}')
    ax[2,0].set_ylabel('Most correct\nsamples', fontsize=16, rotation=0, labelpad=80)

In [ ]:
plot_overview(interp, ['Negative', 'Positive'])

## Sauvegarde du modèle

In [ ]:
# Export de la structure du modèle dans un fichier
learn.export('palmoli_fastai_resnet18.pth')
!ls -lh palmoli_fastai_resnet18.pth

##Test d'une prédiction avec le modèle

In [ ]:
import PIL
#affichage des images sous forme de pixel
test_img = torch.tensor(np.asarray(PIL.Image.open('./leaderboard_holdout_data/img_000012018.jpg')))

In [ ]:
# Prediction sur une image avec le modèle entrainé
results_origin = learn.predict(test_img)

# on recharge le modèle
reloaded_model = load_learner('palmoli_fastai_resnet18.pth')

# Prediction avec le modèle rechargé
results_reload = reloaded_model.predict(test_img)

results_origin, results_reload

In [ ]:
results_origin[2]

## Représentation des images

In [ ]:
test_img.shape

In [ ]:
interp.show_results([263])

In [ ]:
_val, _idx =  interp.top_losses(largest = False)

In [ ]:
_idx[-10:]

In [ ]:
_val[-10:]

In [ ]:
interp.dl.dataset[789][0]

In [ ]:
for idx in ennumerate(_idx):
  print(interp.dl.dataset[int(idx)])